In [20]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [21]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import SemanticSimilarityEvaluator

In [22]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-hugging")

In [23]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [24]:
evaluator = SemanticSimilarityEvaluator()

In [32]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)
query = "Should I use order-none on TailwindCSS to order the grid items first in a different order than they appear in the DOM?"
response = chat_engine.chat(query)
reference = "Use order-{order} to render flex and grid items in a different order than they appear in the DOM."

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'Should I use order-none on TailwindCSS to order the grid items first in a different order than they appear in the DOM?'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai

In [33]:
result = await evaluator.aevaluate(
    response=response.response,
    reference=reference,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [34]:
import pandas as pd
def display_eval_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    score = "{:.1f}".format(eval_result.score)
    eval_df = pd.DataFrame(
        {
            "Response": str(response),
            "Reference": reference,
            "Score": score,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Reference"]
    )
    display(eval_df)

In [35]:
display_eval_df(response, result)

,Response,Reference,Score
0,"Yes, you should use `order-none` on TailwindCSS to order the grid items first in a different order than they appear in the DOM. The `order-none` utility class in TailwindCSS allows you to remove any existing order applied to grid items, so you can then apply a custom order using the `order` utility class.",Use order-{order} to render flex and grid items in a different order than they appear in the DOM.,0.9
